In [2]:
import pandas as pd
import numpy as np


In [3]:
# Add the dataset and predictions together

df = pd.read_csv('synth_data.csv')
preds = pd.read_csv('predictions.csv')
df['predictions'] = preds
df.to_csv('synth_data_preds.csv')


In [4]:
# Add a groundtruth based on the the predictions

df = pd.read_csv('synth_data.csv')
preds = pd.read_csv('predictions.csv')
df['predictions'] = preds
cutoff = df['predictions'].quantile(0.9)
df['actual_outcome'] = (df['predictions'] > cutoff).astype(int)
df.to_csv('altered_data/data_pred_ground_no_alter.csv')

In [5]:
# Add a groundtruth based on the the predictions added some randomizing using a beta distribution

df = pd.read_csv('synth_data.csv')
preds = pd.read_csv('predictions.csv')
df['predictions'] = preds

temp = pd.DataFrame(df['predictions'].clip(0.001, 0.999))

scale = 20  # Higher = less randomness.
alpha = temp['predictions'] * scale
beta = (1 - temp['predictions']) * scale

# Sample from Beta for each row
temp['predictions'] = np.random.beta(alpha, beta)

cutoff = temp['predictions'].quantile(0.9)
df['actual_outcome'] = (temp['predictions'] > cutoff).astype(int)
df.to_csv('altered_data/data_pred_ground_altered_pred.csv')

In [15]:
# same as above but with bias in age and gender

df = pd.read_csv('synth_data.csv')
preds = pd.read_csv('predictions.csv')
df['predictions'] = preds

temp = pd.DataFrame(df['predictions'].clip(0.001, 0.999))

scale = 10  # Higher = less randomness.
gender_bias = .5  # Positive = upward bias for women, negative = downward
age_bias_strength = -0.1  # Each year adds this much bias

alpha = temp['predictions'] * scale
beta = (1 - temp['predictions']) * scale

# Add gender bias
gender_effect = df['persoon_geslacht_vrouw'] * gender_bias
alpha += gender_effect
beta -= gender_effect

# Add age bias
age_center = 50
age_effect = (df['persoon_leeftijd_bij_onderzoek'] - age_center) * age_bias_strength
alpha += age_effect
beta -= age_effect

alpha = np.clip(alpha, 0.001, None)
beta = np.clip(beta, 0.001, None)

# Sample from Beta for each row
temp['predictions'] = np.random.beta(alpha, beta)

cutoff = temp['predictions'].quantile(0.9)
df['actual_outcome'] = (temp['predictions'] > cutoff).astype(int)
df.to_csv('altered_data/data_pred_ground_altered_pred_biased.csv')

In [14]:
pred = .5
scale = 10  # Higher = less randomness.
gender_bias = .5  # Positive = upward bias for women, negative = downward
age_bias_strength = -.1  # Each year adds this much bias

alpha = pred * scale
beta = (1 - pred) * scale

print(alpha, beta)

# Add gender bias
gender_effect = 0 * gender_bias
alpha += gender_effect
beta -= gender_effect

# Add age bias
age_center = 50
age_effect = (20 - age_center) * age_bias_strength
alpha += age_effect
beta -= age_effect

print(alpha, beta)

# persoon_geslacht_vrouw
# persoon_leeftijd_bij_onderzoek

5.0 5.0
8.0 2.0
